In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('heart.csv', )
pd.concat([data.head(), data.tail()])

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1
917,38,M,NAP,138,175,0,Normal,173,N,0.0,Up,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [137]:
from sklearn.preprocessing import OneHotEncoder

def blow_df(df, cat_cols):
    ohe = OneHotEncoder(sparse_output=False)
    print(cat_cols, df)
    encoded_data = ohe.fit_transform(df)
    encoded_df = pd.DataFrame(encoded_data, columns=ohe.get_feature_names_out())
    encoded_df = pd.concat([df.drop(columns=cat_cols), encoded_df], axis=1)
    return encoded_df

In [141]:
for x in cat_data.columns:
    print(x, cat_data[x].value_counts())
    print()

Sex Sex
M    725
F    193
Name: count, dtype: int64

ChestPainType ChestPainType
ASY    496
NAP    203
ATA    173
TA      46
Name: count, dtype: int64

RestingECG RestingECG
Normal    552
LVH       188
ST        178
Name: count, dtype: int64

ExerciseAngina ExerciseAngina
N    547
Y    371
Name: count, dtype: int64

ST_Slope ST_Slope
Flat    460
Up      395
Down     63
Name: count, dtype: int64



In [138]:
cat_cols = list(data.select_dtypes(include=['object']).columns)
num_data = data.select_dtypes(exclude=['object'])
cat_data = data[cat_cols]

# xz = data.iloc[0:1, :]
# display(xz)

def validate_batch(df, batch):
    if (df.nunique().index == batch.nunique().index).mean() < 1:
        return (False, F"column names not equal {df.nunique().index} {batch.nunique().index}")
    if (df.nunique().values == batch.nunique().values).mean() < 1:
        return (False, F"counts not equal {df.nunique().values} {batch.nunique().values}")
    return (True, "ok")

# validate_batch(cat_data, cat_data.iloc[0:200, :])

blow_df(cat_data, cat_cols)


['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']     Sex ChestPainType RestingECG ExerciseAngina ST_Slope
0     M           ATA     Normal              N       Up
1     F           NAP     Normal              N     Flat
2     M           ATA         ST              N       Up
3     F           ASY     Normal              Y     Flat
4     M           NAP     Normal              N       Up
..   ..           ...        ...            ...      ...
913   M            TA     Normal              N     Flat
914   M           ASY     Normal              N     Flat
915   M           ASY     Normal              Y     Flat
916   F           ATA        LVH              N     Flat
917   M           NAP     Normal              N       Up

[918 rows x 5 columns]


,Sex_F,Sex_M,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
914,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
915,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
916,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Пример данных
X = np.array([['cat'], ['dog'], ['bird'], ['cat'], ['dog'], ['fish']])
y = np.array([0, 1, 0, 1, 0, 1])

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Создание и обучение OneHotEncoder на обучающей выборке
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
X_train_encoded = encoder.fit_transform(X_train)

# Применение OneHotEncoder к тестовой выборке
X_test_encoded = encoder.transform(X_test)

print("Обучающая выборка (кодированная):")
print(X_train_encoded)
print("Тестовая выборка (кодированная):")
print(X_test_encoded)
